# **Image Super-Resolution**
**64x64 -> 512x512**

A colab notebook for upscaling 64x64 images to 512x512, using [this](https://github.com/Janspiry/Image-Super-Resolution-via-Iterative-Refinement).


## Setup
---
Instructions: 

*   Turn on hardware acceleration under `Runtime -> Change Runtime Type -> Hardware accelerator -> GPU`
*   Use this command to ensure that the connected machine has a GPU:

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

*    Now, execute each cell sequentially, waiting until each one is done before running the next cell.


### Clone repo, download a pre-trained model, install dependencies.


In [ ]:
%cd /content/
!git clone https://github.com/Janspiry/Image-Super-Resolution-via-Iterative-Refinement.git
!pip install tensorboardx

Now for the download of the pre-trained model file.

Go to the [trained model location](https://drive.google.com/drive/folders/1mCiWhFqHyjt5zE4IdA41fjFwCYdqDzSF) in Google Drive, then right click on the first file and select "Add a shortcut to Drive". Then, choose whichever location to put it in your own Google Drive.

You can now connect this Colab section to your Drive to retrieve the file.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Change the path if you put it elsewhere

In [ ]:
file_location = '/content/drive/MyDrive'

Execute to copy from the Drive to the Colab session

In [ ]:
!cp {file_location}/I830000_E32_gen.pth /content/Image-Super-Resolution-via-Iterative-Refinement/

### Patch config files

Create patchfile



In [ ]:
%%writefile 64_512.patch
--- sr_sr3_64_512_new.json	2021-10-22 16:20:20.901133618 +0000
+++ sr_sr3_64_512.json	2021-10-22 16:20:52.036081672 +0000
@@ -9,8 +9,8 @@
         "tb_logger": "tb_logger",
         "results": "results",
         "checkpoint": "checkpoint",
-        "resume_state": null
-        // "resume_state": "experiments/distributed_high_sr_ffhq_210901_121212/checkpoint/I830000_E32" //pretrain model or training state
+        // "resume_state": null
+        "resume_state": "I830000_E32" //pretrain model or training state
     },
     "datasets": {
         "train": {



Apply patchfile

In [ ]:
!apt-get install dos2unix
!dos2unix sr_sr3_64_512.json
!patch < 64_512.patch

### Prepare Data
Upload 64x64 pixel image(s) to be upscaled

*Click the **browse** button and select the images you would like to upscale.*

Note that the image does not need to be 64x64 since it will be resized in a pre-processing step, however it is better to upload a square image in order to avoid a distorsion.

In [ ]:
%cd /content/Image-Super-Resolution-via-Iterative-Refinement/
!mkdir -p input
%cd /content/Image-Super-Resolution-via-Iterative-Refinement/input/
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  savefile = open(fn, 'wb')
  savefile.write(uploaded[fn])
  print('Successfully uploaded "{}" ({} bytes).'.format(fn, len(uploaded[fn])))
  savefile.close()

Generate neccesary files and directory structure to begin upscaling.  

In [ ]:
%cd /content/Image-Super-Resolution-via-Iterative-Refinement/
!python data/prepare_data.py --path /content/Image-Super-Resolution-via-Iterative-Refinement/input/ --size 64,512 --out ./dataset/celebahq

## Upscaling
---
Note that this may take ~30 minutes or more, depending on which GPU you get.

Output images will be located under ./experiments/\<some folder\>/results

Installing wandb on the Colab machine (by default it is not installed)

In [ ]:
!pip install wandb

In [ ]:
!python infer.py -c /content/Image-Super-Resolution-via-Iterative-Refinement/config/sr_sr3_64_512.json

Let us now visualize the results !

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img

original = plt.imread('input/Poutine6cropped.jpg')
downsized = plt.imread('experiments/distributed_high_sr_ffhq_220301_092743/results/0_5_inf.png')
infered = plt.imread('experiments/distributed_high_sr_ffhq_220301_092743/results/0_5_hr.png')

plt.figure(figsize=(16,8))
plt.subplot(131)
plt.imshow(original)
plt.title('Original high resolution image')
plt.subplot(132)
plt.imshow(downsized)
plt.title('Resized low resolution image')
plt.subplot(133)
plt.imshow(infered)
plt.title('High resolution image infered from the low resolution one')
plt.show()